<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# VAR Models
_Author: Matt Brems_

---

### Learning Objectives
_By the end of the lesson, students should be able to:_
- Describe univariate and multivariate time series.
- Identify the advantages of working with multivariate time series.
- Define VAR models.
- Understand and test for the assumptions of VAR models.
- Fit, generate predictions from, and evaluate VAR models.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

## Extensions to ARIMA Model

As we discussed before, the ARIMA model is a very flexible model that can be extended in multiple ways.
- Seasonal ARIMA: An ARIMA model that can account for seasonal fluctuations and differences.
- ARIMA with eXogenous Predictors: An ARIMA model that can include eXogenous predictors (independent X variables).
- Vector ARIMA models: An ARIMA model that can handle multivariate time series.

## Multivariate Time Series Models

**Univariate** time series methods are those that deal with forecasting one variable into the future.

$$
\begin{eqnarray*}
Y^{(d)}_t = AR(p) + MA(q) + \text{seasonal component} + X_{1, t-1} + X_{2, t-1} + X_{3, t-1}
\end{eqnarray*}
$$

In this situation, we're taking three independent variables ($X_1$, $X_2$, $X_3$) and using them to predict $Y$. However, if $X_1$, $X_2$, $X_3$ are related to $Y$, does it make sense to forecast $X_1$, $X_2$, $X_3$ forward as well?

**Multivariate** time series methods are those that deal with forecasting multiple variable into the future.

Let's relabel $X_1$, $X_2$, $X_3$ as $Y_2$, $Y_3$, $Y_4$.

$$
\begin{eqnarray*}
\text{original series: } Y^{(d)}_{1,t} &=& AR(p) + MA(q) + \text{seasonal component} + Y_{2, t-1} + Y_{3, t-1} + Y_{4, t-1} \\
\\
Y^{(d)}_{2,t} &=& AR(p) + MA(q) + \text{seasonal component} + Y_{1, t-1} + Y_{3, t-1} + Y_{4, t-1} \\
\\
Y^{(d)}_{3,t} &=& AR(p) + MA(q) + \text{seasonal component} + Y_{1, t-1} + Y_{2, t-1} + Y_{4, t-1} \\
\\
Y^
{(d)}_{4,t} &=& AR(p) + MA(q) + \text{seasonal component} + Y_{1, t-1} + Y_{2, t-1} + Y_{3, t-1} \\
\end{eqnarray*}
$$

This makes the most sense as a strategy when our variables are all related. 
- $Y_1$ is related to $Y_2$, $Y_3$, $Y_4$,
- $Y_2$ is related to $Y_1$, $Y_3$, $Y_4$,
- and so on.

The most common method of tackling this is to use VAR models.

## VAR Time Series Models

VAR time series models are **vector autoregressive models**.
- Rather than regressing one time series $Y_t$ on lagged versions of itself and on lagged versions of independent variables $X$, we will take all of our variables $Y_{1,t}, Y_{2,t}, Y_{3,t}, \ldots$ and regress them on one another simultaneously.
- This allows us to forecast forward many variables simultaneously.

<details><summary>What are examples of variables we may want to forecast forward together?</summary>
    
_(Answers may vary.)_

- Prevalence of multiple pollutants (like chemicals from factories).
- Exchange rates between countries. (e.g. USD to EUR, EUR to CHF, CHF to GBP, GBP to USD.)
- Macroeconomic indices (like GDP and unemployment rate) that may influence one another.
</details>

In [ ]:
# Import VAR
from statsmodels.tsa.api import VAR

In [ ]:
# Load up the macroeconomic data.
df =

> We'll be using [U.S. Macroeconomic data](https://www.statsmodels.org/0.6.1/datasets/generated/macrodata.html) that is available via the `statsmodels` package. This is quarter-level data from 1959 to 2009.
- `year`: the year the data was measured.
- `quarter`: the quarter the data was measured.
- `realgdp`: real gross domestic product (in billions, in 2005 US dollars).
- `pop`: total population measured at the end of the quarter (includes all ages and overseas armed forces members).
- `unemp`: seasonally adjusted unemployment rate (in percent).

In [ ]:
df.head()

In [ ]:
# Create an array called "dates" based on "year" and "quarter."
dates =

# Create a vector called "quarterly" combining year and quarter together.
quarterly = 

# Import dates_from_str and convert "quarterly" into dates.
from statsmodels.tsa.base.datetools import dates_from_str
quarterly = dates_from_str(quarterly)

In [ ]:
# Select only the realgdp, pop, unemp columns.
df = df[['realgdp','pop','unemp']]

# Set index to be "quarterly."
df.index =

df.head()

In [ ]:
# Code modified from code written by Matthew Garton.

def plot_series(df, cols=None, title='Title', xlab=None, ylab=None, steps=1):
    
    # Set figure size to be (18, 9).
    plt.figure(figsize=(18,9))
    
    # Iterate through each column name.
    for col in cols:
            
        # Generate a line plot of the column name.
        # You only have to specify Y, since our
        # index will be a datetime index.
        plt.plot(df[col])
        
    # Generate title and labels.
    plt.title(title, fontsize=26)
    plt.xlabel(xlab, fontsize=20)
    plt.ylabel(ylab, fontsize=20)
    
    # Enlarge tick marks.
    plt.yticks(fontsize=18)
    plt.xticks(df.index[0::steps], fontsize=18);

In [ ]:
# Plot our real GDP data.
plot_series()

In [ ]:
# Plot our population data.


In [ ]:
# Plot our unemployment data.


### Model Fitting Process
1. Confirm stationarity of the data.
2. Train/test split.
3. Determine correct lag order $p$.
4. Fit model.
5. Generate forecasts.
6. Evaluate model and forecasts (if possible).

#### 1. Confirm stationarity of the data.
Vector autoregressive models require stationarity in order for us to fit them!

<details><summary>How would you describe stationarity?</summary>

- When our time series does not have systematic changes over time.
- Constant mean, correlation that only depends on lag (and not point in time).
</details>

<details><summary>What do we use to check whether or not a time series is stationary?</summary>

- Augmented Dickey-Fuller test!
</details>

In [ ]:
# Code written by Joseph Nelson.

def interpret_dftest(dftest):
    dfoutput = pd.Series(dftest[0:2], index=['Test Statistic','p-value'])
    return dfoutput

In [ ]:
# Run ADF test on the original Real GDP data.


<details><summary>Based on the results here, what action should we take?</summary>

- Difference our data.
</details>

In [ ]:
# Create column.


In [ ]:
df.head()

Assuming $\alpha=0.01$, take two minutes to achieve stationarity for `pop` and `unemp`. If needed, create the columns we want to model.

#### 2. Train/test split.

In [ ]:
# Subset our data.
df = df[['first_diff_realgdp', 'second_diff_pop', 'first_diff_unemp']]

# Let's get rid of rows containing missing values.
df.dropna(inplace = True)

In [ ]:
# What am I missing?

train, test = train_test_split(df,
                               test_size = 0.25)

#### 3. Determine correct lag order $p$.

We can check out autocorrelation plots and partial autocorrelation plots to attempt to figure out how many previous values of **each variable** we want in our model.

Suppose I select $p=2$.

$$
\begin{eqnarray*}
Y^{(d)}_{1,t} &=& \beta_0 + \beta_1Y^{(d)}_{1,t-1} + \beta_2Y^{(d)}_{1,t-2} + \beta_3Y^{(d)}_{2,t-1} + \beta_4Y^{(d)}_{2,t-2} + \beta_5Y^{(d)}_{3,t-1} + \beta_6Y^{(d)}_{3,t-2} \\
\\
Y^{(d)}_{2,t} &=& \gamma_0 + \gamma_1Y^{(d)}_{1,t-1} + \gamma_2Y^{(d)}_{1,t-2} + \gamma_3Y^{(d)}_{2,t-1} + \gamma_4Y^{(d)}_{2,t-2} + \gamma_5Y^{(d)}_{3,t-1} + \gamma_6Y^{(d)}_{3,t-2} \\
\\
Y^{(d)}_{3,t} &=& \delta_0 + \delta_1Y^{(d)}_{1,t-1} + \delta_2Y^{(d)}_{1,t-2} + \delta_3Y^{(d)}_{2,t-1} + \delta_4Y^{(d)}_{2,t-2} + \delta_5Y^{(d)}_{3,t-1} + \delta_6Y^{(d)}_{3,t-2}
\\
\end{eqnarray*}
$$

However, we can automate the selection of $p$ using a metric called the Akaike information criterion, or the AIC.

##### AIC

The AIC is a metric that is commonly used for time series models or in more "statistics-oriented" fields.
- Recall that models are just simplifications of reality. AIC attempts to measure how much information we lose when we simplify reality with a model.
- The lower the AIC, the better!
- More details can be found at the [Wikipedia page](https://en.wikipedia.org/wiki/Akaike_information_criterion).

We can actually find a good value of $p$ when we fit our model!

#### 4. Fit model.

In [ ]:
# Instantiate a VAR model. Remember that we pass
# our data in during instantiation in statsmodels!
model =

In [ ]:
# Fit our model and use AIC to select the value of p.

ts_model = model.fit(, # what is the largest possible value of p?
                     )   # what "information criterion" (ic) will we use to decide what's "best?"

In [ ]:
# What is the order of our autoregressive model? 


In [ ]:
# Check out the summary of our model!


In [ ]:
# Plot our training data.
;

#### 5. Generate forecasts.

In [ ]:
# Plot the forecast looking 3 steps ahead.
ts_model.plot_forecast(3);

In [ ]:
# Plot the forecast looking 50 steps ahead.
;

In [ ]:
# Generate a forecast one step ahead.
ts_model.forecast(train.values, 1)

In [ ]:
# Generate a forecast five steps ahead.


In [ ]:
# Generate a forecast that matches our testing set.


In [ ]:
# See the values of our test set.


#### 6. Evaluate model (and forecasts, if possible).

In [ ]:
# Let's use MSE to evaluate our models.

# Save our forecast as forecast.
forecast = ts_model.forecast(train.values, len(test))

# Instantiate MSE values at 0.
mse_gdp = 0
mse_pop = 0
mse_unemp = 0

# Loop through each forecasted time point.
for time in range(len(test)):
    
    # Calculate (expected - observed) ** 2 and add to MSE.
    mse_gdp += (forecast[time][0] - test.values[time][0]) ** 2
    mse_pop += (forecast[time][1] - test.values[time][1]) ** 2
    mse_unemp += (forecast[time][2] - test.values[time][2]) ** 2

# Divide SSE to get MSE.
mse_gdp /= len(test)
mse_pop /= len(test)
mse_unemp /= len(test)
    
# Generate output.    
print(f'The test MSE on the Real GDP data is: {round(mse_gdp, 4)}')
print(f'The test MSE on the population data is: {round(mse_pop, 4)}')
print(f'The test MSE on the unemployment data is: {round(mse_unemp, 4)}')